# Sw00pGenerator3000 (SG3K) - Dataset Creation with FlySight

What’s swooping without FlySight? FlySight is not just an ordinary GPS; it's specifically designed for wingsuit pilots and introduces groundbreaking features. It provides real-time audible feedback on glide ratio, horizontal, and vertical speeds, enhancing your flying experience. For more information about FlySight, visit this [link](https://github.com/flysight/flysight). 

Now, let's discuss FlySight 2, a complete redesign that builds upon the success of its predecessor. FlySight 2 features real-time audible feedback through tones and speech, allowing you to review your jump data using the same applications you loved in FlySight 1. This new version includes several enhancements, such as:

- **Water-resistant case (IP67)**
- **More powerful processor with Bluetooth Low Energy**
- **Additional sensors**: acceleration, rotation, orientation, pressure, humidity, and temperature for comprehensive flight analysis

For those interested in the technical details, let's compare the track data from FlySight Gen1, Gen2, and the newly created dataset. The "Dataset" class generates a fresh dataset from the FlySight device, which can be stored in InfluxDB to preserve jump information. The variables in the newly generated dataset have the following significance:

<table>
<tr><th align="center">Flysight Gen1 - Track data</th><th align="center">Flysight Gen2 - Track data</th><th align="center">Created Dataset</th></tr>

<tr><td>

| **Name**   | **Unit**  | **Description**                               |
|:-----------|:----------|:----------------------------------------------|
| time       | datetime  | Time in ISO8601 format                        |
| lat        | deg       | Latitude                                      |
| lon        | deg       | Longitude                                     |
| hMSL       | m         | Height above mean sea level                   |
| velN       | m/s       | NED north velocity                            |
| velE       | m/s       | NED east velocity                             |
| velD       | m/s       | NED down velocity                             |
| hAcc       | m         | Horizontal Accuracy Estimate                  |
| vAcc       | m         | Vertical Accuracy Estimate                    |
| sAcc       | m/s       | Speed Accuracy Estimate                       |
| heading    | deg       | Heading of motion 2-D                         |
| cAcc       | deg       | Course / Heading Accuracy Estimate            |
| gpsFix     | -         | GPSfix Type (3 = 3D)                          |
| numSV      | -         | # of satellites following.                    |

</td><td>

| **Name**   | **Unit**  | **Description**                               |
|:-----------|:----------|:----------------------------------------------|
| time       | datetime  | Time in ISO8601 format                        |
| lat        | deg       | Latitude                                      |
| lon        | deg       | Longitude                                     |
| hMSL       | m         | Height above mean sea level                   |
| velN       | m/s       | NED north velocity                            |
| velE       | m/s       | NED east velocity                             |
| velD       | m/s       | NED down velocity                             |
| hAcc       | m         | Horizontal Accuracy Estimate                  |
| vAcc       | m         | Vertical Accuracy Estimate                    |
| sAcc       | m/s       | Speed Accuracy Estimate                       |
| numSV      | -         | # of satellites following.                    |
| -          | -         | -                                             |
| -          | -         | -                                             |
| -          | -         | -                                             |

</td><td>

| **Name**           | **Description**                               |
|:-------------------|:----------------------------------------------|
| timestamp          | year-month-day hour:minute:second:millisecond |
| time_sec           | Time (s)                                      |
| lat                | Latitude                                      |
| lon                | Longitude                                     |
| elevation          | Elevation                                     |
| horz_distance_ft   | Horizontal Distance feet                      |
| horz_distance_m    | Horizontal Distance meter                     |
| x_axis_distance_ft | X Axis Distance feet                          |
| x_axis_distance_m  | X Axis Distance meter                         |
| y_axis_distance_ft | Y Axis Distance feet                          |
| y_axis_distance_m  | Y Axis Distance meter                         |
| vert_speed_mph     | Vertical speed miles per hour                 |
| horz_speed_mph     | Horizontal speed miles per hour               |
| vert_speed_km/u    | Vertical speed kilometers per hour            |
| horz_speed_km/u    | Horizontal speed kilometers per hour          |
| dive_angle         | Dive angle                                    |
| name               | Name of dataset                               |
| user_id            | User id that generated the data               |

</td></tr>
</table>

The FlySight2 includes supplementary sensor data, comprising the following information:

- **BARO:** time(s), pressure(Pa), temperature(deg C)
- **HUM:** time(s), humidity(percent), temperature(deg C)
- **MAG:** time(s), x(gauss), y(gauss), z(gauss), temperature(deg C)
- **IMU:** time(s), wx(deg/s), wy(deg/s), wz(deg/s), ax(g), ay(g), az(g), temperature(deg C)
- **TIME:** time(s), tow(s), week
- **VBAT:** time(s), voltage(volt)

It's important to note that as of now, this additional data is not utilized in the current application. For the comprehensive technical specifications FlySight 2, refer to the official documentation [link](https://content.u-blox.com/sites/default/files/products/documents/u-blox6_ReceiverDescrProtSpec_%28GPS.G6-SW-10018%29_Public.pdf). Get ready to elevate your swooping experience with the SG3K and FlySight 2 - the perfect duo for soaring to new heights!

### Adjust the settings to suit your preference

In [1]:
swoop_file = '../data/jumps/flysight_gen2/katana120/24-10-05/09-00-41/TRACK.CSV'
dropzone_elevation = None # If set to None elevation will be dynamically set, Elevation for MOORSELEEUHH 20.12

### Imports

In [2]:
import utm
import pyproj
import textwrap
import numpy as np
import pandas as pd
import geopy.distance
from math import sqrt, degrees, atan, pi, radians, sin, cos, atan2

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Dataset Creation

The "DatasetService" class creates a new dataset from the FlySight device, allowing the jump information to be stored in InfluxDB for preservation.

In [3]:
METERS_TO_FEET = 3.280839895
FEET_TO_METERS = 1 / METERS_TO_FEET
MPERSEC_TO_MPH = 2.236936
MPERSEC_TO_KPH = 3.6

def meters_to_feet(meters):
    """Converts meters to feet."""
    return meters * METERS_TO_FEET

def feet_to_meters(feet):
    """Converts feet to meters."""
    return feet * FEET_TO_METERS

def mps_to_mph(mps):
    """Converts meters per second to miles per hour."""
    return mps * MPERSEC_TO_MPH

def mps_to_kph(mps):
    """Converts meters per second to kilometers per hour."""
    return mps * MPERSEC_TO_KPH

def calc_horizontal_speed(n, e):
    """Calculates horizontal speed from north and east velocity components."""
    return sqrt((n**2) + (e**2))

def calc_dive_angle(v_speed, h_speed):
    """Calculates the dive angle based on vertical and horizontal speeds."""
    try:
        return degrees(atan(v_speed / h_speed))
    except ZeroDivisionError:
        return 0.0

def calc_distance_geo(metric, lat1, lat2, lon1, lon2):
    """Calculates the distance between two geo-coordinates in the specified metric ('m' for meters, 'ft' for feet)."""
    coordinates_1 = (lat1, lon1)
    coordinates_2 = (lat2, lon2)
    if metric == 'ft':
        return geopy.distance.geodesic(coordinates_1, coordinates_2).miles * 5280
    elif metric == 'm':
        return geopy.distance.geodesic(coordinates_1, coordinates_2).kilometers * 1000
    else:
        raise ValueError(f"Invalid metric: {metric}. Use 'm' for meters or 'ft' for feet.")

def to_utm(lat, lon):
    """Converts latitude and longitude to UTM (Universal Transverse Mercator) coordinates."""
    try:
        zone = utm.from_latlon(lat[0], lon[0])[2]
        p = pyproj.Proj(proj='utm', zone=zone, ellps='WGS84')
        return p(lon, lat)
    except Exception as e:
            raise ValueError(f"Error converting to UTM: {e}")

def calc_axis_distance(metric, lat, lon):
    """Calculates the axis distance (x, y) in either meters or feet based on latitudes and longitudes."""
    x, y = to_utm(lat, lon)
    if metric == 'ft':
        return [meters_to_feet(x_cor) for x_cor in x], [meters_to_feet(y_cor) for y_cor in y]
    elif metric == 'm':
        return x, y
    else:
        raise ValueError(f"Invalid metric: {metric}. Use 'm' for meters or 'ft' for feet.")
        

class DatasetService:
    """Service to process flight data stored in CSV format."""
    def __init__(self, file, filename='v1', dropzone_elevation=None, user_id=None):
        self.file = file
        self.filename = filename
        self.dropzone_elevation = dropzone_elevation
        self.user_id = user_id
        self.df = self.read_csv()

    def read_csv(self):
        """Reads the CSV file and handles various formats."""
        try:
            # read flysight gen 1
            df = pd.read_csv(self.file, skiprows=[1])
            return df
        except Exception as e:
            try:
                # read flysight gen 2
                df = pd.read_csv(self.file, skiprows=7, header=None)
                df = df.drop(df.columns[0], axis=1)
                df.columns = ['time', 'lat', 'lon', 'hMSL', 'velN', 'velE', 'velD', 'hAcc', 'vAcc', 'sAcc', 'numSV']
                return df
            except Exception as e:
                print("NO FLYSIGHT TRACK DATA DETECTED!!")
                return None
 
    def get_total_seconds(self):
        """Calculates total seconds for each time point."""
        datetimes = [pd.to_datetime(d, format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce') for d in self.df.time]
        l = []
        for i, d in enumerate(datetimes):
            duration = datetimes[i] - datetimes[0]
            l.append(duration.total_seconds())
        return l

    def get_elevation(self):
        """Calculates dynamic or fixed elevation depending on dropzone elevation."""
        ground_elevation = meters_to_feet(self.df.hMSL.iloc[-1])
        return [meters_to_feet(h) - ground_elevation for h in self.df.hMSL] if self.dropzone_elevation is None else \
               [meters_to_feet(h) - meters_to_feet(self.dropzone_elevation) for h in self.df.hMSL]

    def get_vertical_speed(self, metric):
        """Calculates vertical speed based on the given metric (mph or km/u)."""
        if metric == 'mph':
            return [mps_to_mph(m) for m in self.df.velD]
        elif metric == 'km/u':
            return [mps_to_kph(m) for m in self.df.velD]
        raise ValueError(f"Invalid metric: {metric}")

    def get_horizontal_speed(self, metric):
        """Calculates horizontal speed based on the given metric (mph or km/u)."""
        speed = [calc_horizontal_speed(self.df.velN[i], self.df.velE[i]) for i in range(0, len(self.df))]
        if metric == 'mph':
            return [mps_to_mph(s) for s in speed]
        elif metric == 'km/u':
            return [mps_to_kph(s) for s in speed]
        raise ValueError(f"Invalid metric: {metric}")

    def get_dive_angle(self, v_speed, h_speed):
        """Calculates dive angle from vertical and horizontal speeds."""
        return [calc_dive_angle(v_speed[i], h_speed[i]) for i in range(0, len(self.df))]

    def get_horizontal_distance(self, metric):
        """Calculates cumulative horizontal distance."""
        lis, dis = [0], 0.0
        for i in range(0, len(self.df) - 1):
            dis += calc_distance_geo(metric, self.df.lat[i], self.df.lat[i + 1], self.df.lon[i], self.df.lon[i + 1])
            lis.append(dis)
        return lis

    def get_axis_distance(self, metric, axis):
        """Calculates distance along x or y axis."""
        x, y = calc_axis_distance(metric, self.df.lat, self.df.lon)
        axis_value = x if axis == 'x' else y
        lis, dis = [0], 0.0
        for i in range(0, len(axis_value) - 1):
            dis += axis_value[i + 1] - axis_value[i]
            lis.append(dis)
        return lis

    def create_jump_data(self):
        """Creates a DataFrame with all processed data related to the jump."""
        if self.df is None:
            return None

        return pd.DataFrame({
            'timestamp': pd.to_datetime(self.df.time, format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce'),
            'time_sec': np.array(self.get_total_seconds()),
            'lat': self.df.lat,
            'lon': self.df.lon,
            'elevation': self.get_elevation(),
            'horz_distance_ft': self.get_horizontal_distance('ft'),
            'horz_distance_m': self.get_horizontal_distance('m'),
            'x_axis_distance_ft': self.get_axis_distance('ft', 'x'),
            'x_axis_distance_m': self.get_axis_distance('m', 'x'),
            'y_axis_distance_ft': self.get_axis_distance('ft', 'y'),
            'y_axis_distance_m': self.get_axis_distance('m', 'y'),
            'vert_speed_mph': self.get_vertical_speed('mph'),
            'horz_speed_mph': self.get_horizontal_speed('mph'),
            'vert_speed_km/u': self.get_vertical_speed('km/u'),
            'horz_speed_km/u': self.get_horizontal_speed('km/u'),
            'dive_angle': self.get_dive_angle(self.get_vertical_speed('mph'), self.get_horizontal_speed('mph')),
            'name': self.get_name(),
            'user_id': self.user_id
        })
    
    def create_pond_data(self):
        """Creates a DataFrame for pond data, focusing on horizontal speeds."""
        if self.df is None:
            return None
        
        return pd.DataFrame({
            'timestamp': pd.to_datetime(self.df.time),
            'time_sec': np.array(self.get_total_seconds()),
            'lat': self.df.lat,
            'lon': self.df.lon,
            'horz_speed_mph': self.get_horizontal_speed('mph'),
            'horz_speed_km/u': self.get_horizontal_speed('km/u')
        })

    def get_name(self):
        """Generates a name for the dataset."""
        return pd.to_datetime(self.df.time[0]).strftime("D%m-%d-%YT%H%M") + self.filename

    def save(self, path):
        """Saves the dataset to a CSV file."""
        dataframe = self.create()
        dataframe.to_csv(os.path.join(path, self.get_name() + '.csv'), index=False)

    def __str__(self):
        return f'{self.get_name()}'

jump_df = DatasetService(swoop_file, dropzone_elevation=dropzone_elevation).create_jump_data()
if jump_df is not None:
    display(jump_df.head())

,timestamp,time_sec,lat,lon,elevation,horz_distance_ft,horz_distance_m,x_axis_distance_ft,x_axis_distance_m,y_axis_distance_ft,y_axis_distance_m,vert_speed_mph,horz_speed_mph,vert_speed_km/u,horz_speed_km/u,dive_angle,name,user_id
0,2024-10-05 09:00:41.200,0.0,50.848586,3.145033,111.748688,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.648711,0.890851,-1.044,1.433686,-36.061839,D10-05-2024T0900v1,None
1,2024-10-05 09:00:41.400,0.2,50.848568,3.145008,97.614829,8.697624,2.651036,-5.599382,-1.706692,-6.650959,-2.027212,-0.559234,0.918234,-0.900,1.477755,-31.342749,D10-05-2024T0900v1,None
2,2024-10-05 09:00:41.600,0.4,50.848570,3.144983,80.108268,14.483801,4.414663,-11.328856,-3.453035,-5.859570,-1.785997,0.246063,1.176892,0.396,1.894024,11.809209,D10-05-2024T0900v1,None
3,2024-10-05 09:00:41.800,0.6,50.848583,3.144956,63.175853,22.386243,6.823327,-17.735638,-5.405823,-1.238756,-0.377573,0.044739,0.044739,0.072,0.072000,45.000000,D10-05-2024T0900v1,None
4,2024-10-05 09:00:42.000,0.8,50.848586,3.144926,49.419291,29.389209,8.957831,-24.620857,-7.504437,0.024654,0.007515,0.626342,0.192429,1.008,0.309684,72.921686,D10-05-2024T0900v1,None
